In [ ]:
# ==============================================================================
# CELL 1: BOOTSTRAP - INTELLIGENT ENVIRONMENT SETUP
# ==============================================================================
# This cell checks if the required dependencies are already installed. If not,
# it downloads and runs a setup script to prepare the Colab environment.

import sys
import subprocess
import os
import shutil

# --- Configuration ---
# Set to True to install the full OpenStudio CLI (required for running Measures)
INSTALL_CLI = False
# Set the branch of the toolkit to use ('main' for stable, 'dev' for development)
TOOLKIT_BRANCH = 'main'

# --- Verification and Execution Logic ---
try:
    # Attempt to import the core libraries.
    import openstudio
    import openstudio_toolkit
    
    # If INSTALL_CLI is True, also check for the command line interface.
    if INSTALL_CLI and not shutil.which('openstudio'):
        # This will trigger the installation if the CLI is missing but the library is present.
        raise ImportError("OpenStudio CLI not found")

    print("✅ Environment is already set up. Skipping installation.")

except ImportError:
    print("⏳ Required dependencies not found. Proceeding with setup...")
    
    bootstrap_script = 'bootstrap.py'
    # Download the bootstrap script if it doesn't exist OR if it's an empty file.
    if not os.path.exists(bootstrap_script) or os.path.getsize(bootstrap_script) == 0:
        print("--> Downloading bootstrap script...")
        bootstrap_url = f"https://raw.githubusercontent.com/roruizf/OpenStudio-Toolkit/{TOOLKIT_BRANCH}/{bootstrap_script}"
        os.system(f"wget -q -O {bootstrap_script} {bootstrap_url}")

    # Final check to ensure the downloaded script is not empty before running
    if os.path.exists(bootstrap_script) and os.path.getsize(bootstrap_script) > 0:
        print("--> Running environment setup...")
        cli_flag = "--install-cli" if INSTALL_CLI else ""
        os.system(f"python {bootstrap_script} --branch {TOOLKIT_BRANCH} {cli_flag}")
        
        print("\n💥 KERNEL RESTART is likely required for the changes to take effect.")
        print("Please go to 'Runtime' > 'Restart session' in the menu, then run the next cell.")
    else:
        print("❌ ERROR: Failed to download the bootstrap script. Please check the URL and your connection.")

In [ ]:
# ==============================================================================
# CELL 2: RUN WORKFLOW
# ==============================================================================
# IMPORTANT: Run this cell AFTER restarting the kernel from Cell 1 if needed.

import os
import sys

# --- Imports from the Toolkit ---
from openstudio_toolkit.utils.notebook_helpers import get_osm_path
from openstudio_toolkit.utils.osm_utils import load_osm_file_as_model, save_model_as_osm_file
# Import both tasks as requested
from openstudio_toolkit.tasks.model_setup import copy_subsurfaces_geometry, rename_subsurfaces_based_on_space_and_surface_names, rename_surfaces_based_on_space_names


# --- User Inputs for File Access ---
USE_GDRIVE = False
# --- Path to the REFERENCE model (source of geometry)
GDRIVE_REF_OSM_PATH = '/content/drive/MyDrive/path/to/your/RefModel.osm'
LOCAL_REF_OSM_PATH = '/mnt/g/My Drive/R2F-EES/Deliver Services/Projects/Dosta Tec/CCB43&FPM4/Working Files/LEED v4/FPM4-029.osm'
# --- Path to the TARGET model (the one to be modified)
GDRIVE_TARGET_OSM_PATH = '/content/drive/MyDrive/path/to/your/TargetModel.osm'
LOCAL_TARGET_OSM_PATH = '/mnt/g/My Drive/R2F-EES/Deliver Services/Projects/Dosta Tec/CCB43&FPM4/Working Files/LEED v4/FPM4-029-BSLN.osm'

# --- Get File Paths ---
try:
    ref_osm_path, _ = get_osm_path(
        use_gdrive=USE_GDRIVE, gdrive_path=GDRIVE_REF_OSM_PATH, local_path=LOCAL_REF_OSM_PATH)
    target_osm_path, _ = get_osm_path(
        use_gdrive=USE_GDRIVE, gdrive_path=GDRIVE_TARGET_OSM_PATH, local_path=LOCAL_TARGET_OSM_PATH)
except (ValueError, FileNotFoundError) as e:
    print(e)
    ref_osm_path, target_osm_path = None, None

# --- Main Execution Block ---
if ref_osm_path and target_osm_path and os.path.exists(ref_osm_path) and os.path.exists(target_osm_path):

    file_name = os.path.basename(target_osm_path)
    output_file_name = f"{os.path.splitext(file_name)[0]}_geometry_updated.osm"
    output_osm_path = os.path.join(os.getcwd(), output_file_name)
    print(f"INFO: Output will be saved to: {output_osm_path}")

    # 1. Load Models
    print(f"\nINFO: Loading REFERENCE model from: {ref_osm_path}")
    reference_model = load_osm_file_as_model(ref_osm_path)
    print(f"INFO: Loading TARGET model from: {target_osm_path}")
    target_model = load_osm_file_as_model(target_osm_path)

    # 2. Rename Subsurfaces in Both Models to Ensure Consistency
    print("\n--- Standardizing Subsurface Names ---")
    print("INFO: Renaming subsurfaces in REFERENCE model...")
    # The run() function modifies the model in place and returns it
    reference_model = rename_surfaces_based_on_space_names.run(reference_model)
    reference_model = rename_subsurfaces_based_on_space_and_surface_names.run(reference_model)
    print("INFO: Renaming subsurfaces in TARGET model...")
    target_model = rename_surfaces_based_on_space_names.run(target_model)
    target_model = rename_subsurfaces_based_on_space_and_surface_names.run(target_model)
    print("--------------------------------------")

    # 3. Validate Models for Geometry Copy Task
    print("\n--- Validating Models for Geometry Copy Task ---")
    report = copy_subsurfaces_geometry.validator(
        reference_model=reference_model,
        target_model=target_model
    )
    for msg in report['messages']:
        print(f"- {msg}")
    print("--------------------------------------------")

    # 4. Run Geometry Copy Task and Save
    if report['status'] == 'READY':
        print("\n--- Running Geometry Copy Task ---")
        modified_model = copy_subsurfaces_geometry.run(
            reference_model=reference_model,
            target_model=target_model
        )
        save_model_as_osm_file(modified_model, output_osm_path)
        print(f"\n✅ Model saved successfully.")

        # 5. Verification Step
        print("\n--- Verifying Results ---")
        verified_model = load_osm_file_as_model(output_osm_path)

        # Pick one subsurface to check by name
        # !!! IMPORTANT: CHANGE THIS to a valid (and now renamed) subsurface name in your model !!!
        SUBSURFACE_TO_CHECK = "Specify_Subsurface_Name_Here"

        ref_sub = reference_model.getSubSurfaceByName(SUBSURFACE_TO_CHECK)
        mod_sub = verified_model.getSubSurfaceByName(SUBSURFACE_TO_CHECK)

        if ref_sub.is_initialized() and mod_sub.is_initialized():
            print(f"\n--- Comparing vertices for '{SUBSURFACE_TO_CHECK}' ---")
            print("Reference model vertices:")
            for v in ref_sub.get().vertices():
                print(f"  ({v.x():.3f}, {v.y():.3f}, {v.z():.3f})")

            print("\nModified model vertices:")
            for v in mod_sub.get().vertices():
                print(f"  ({v.x():.3f}, {v.y():.3f}, {v.z():.3f})")
            print("-----------------------------------------")
        else:
            print(f"\n❌ VERIFICATION FAILED: Could not find subsurface '{SUBSURFACE_TO_CHECK}'. Please update the name.")
    else:
        print("\n❌ Task execution aborted due to validation status not being 'READY'.")
else:
    if not ref_osm_path or not os.path.exists(ref_osm_path):
        print(f"❌ ERROR: The specified REFERENCE OSM file path does not exist or was not provided.")
    if not target_osm_path or not os.path.exists(target_osm_path):
        print(f"❌ ERROR: The specified TARGET OSM file path does not exist or was not provided.")


In [ ]:
# --- DEBUG: Find mismatched subsurface names ---
ref_names = {s.name().get() for s in reference_model.getSubSurfaces()}
target_names = {s.name().get() for s in target_model.getSubSurfaces()}

# Find names that are in the reference model but not in the target
in_ref_not_in_target = ref_names.difference(target_names)
# Find names that are in the target model but not in the reference
in_target_not_in_ref = target_names.difference(ref_names)

if in_ref_not_in_target or in_target_not_in_ref:
    print("❌ ERROR: Mismatch in subsurface names found!")
    if in_ref_not_in_target:
        # Sort the set alphabetically before printing
        print(f"--> Names found ONLY in the REFERENCE model: {sorted(in_ref_not_in_target)}")
    if in_target_not_in_ref:
        # Sort the set alphabetically before printing
        print(f"--> Names found ONLY in the TARGET model: {sorted(in_target_not_in_ref)}")
else:
    print("✅ SUCCESS: All subsurface names match perfectly between the models.")